In [1]:
import json, os, re, sqlite3, thread, urllib, urllib2
from shapely.geometry import Point
import collections


# Google geocoding API to geocode an address
# Assumes key is in google_api_key.txt
# Use as second level lookup only for those which fail census lookup

re_noalnum = re.compile('^([^\w]+)$')

def geocode_address_google(address):
    if(pandas.isnull(address) or re_noalnum.match(address)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(address)
        return None

    try:
        geocode_address_google.conn
    except:
        geocode_address_google.conn = sqlite3.connect('geocoding_cache')
        geocode_address_google.cur = geocode_address_google.conn.cursor()
        geocode_address_google.cur.execute(('CREATE TABLE IF NOT EXISTS kvs'
                                     ' (key PRIMARY KEY, value)'
                                     ' WITHOUT ROWID;'))
        geocode_address_google.conn.commit()

    geocode_address_google.cur.execute('SELECT value FROM kvs WHERE key=?', (address,))
    rows = geocode_address_google.cur.fetchall()
    if rows:
        #print "Google: Found %r in cache" % (address)
        return json.loads(rows[0][0])
        
    #print "Google: %r not found in cache, fetching" % (address)

    api_key = open('google-api-key-do-not-commit.txt').read()
    payload = {'address':address, 'key':api_key}
    result = json.load(urllib2.urlopen('https://maps.googleapis.com/maps/api/geocode/json?%s' % urllib.urlencode(payload)))
    
    geocode_address_google.cur.execute(('INSERT OR REPLACE INTO kvs (key, value)'
                                 ' VALUES (?, ?);'),
                                 (address, json.dumps(result)))
    geocode_address_google.conn.commit()
    
    # If the result succeeded and generated a formatted address, cache under that key too
    try:
        if(result['status']=='OK'):
            if(len(result['results'])== 1):
                canonical = result['results'][0]['formatted_address']
                #print "Google: Storing canonical addr of %r for %r" % (canonical, address)
                geocode_address_google.cur.execute(('INSERT OR REPLACE INTO kvs (key, value)'
                                                     ' VALUES (?, ?);'),
                                                     (canonical, json.dumps(result)))
                geocode_address_google.conn.commit()
            else:
                print "Google: Fetch %r seemed to succeed, but results empty or non-unique" % (address)

        else:
            print "Google: Fetch %r failed" % (address)

    except:
        #pass
        raise
    
    return result

def get_canonical_address_google(raw_addr):
    if(pandas.isnull(raw_addr) or re_noalnum.match(raw_addr)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(raw_addr)
        return None

    try:
        result=geocode_address_google(raw_addr)
        if(result['status']=='OK' and len(result['results'])== 1):
            return(result['results'][0]['formatted_address'])
        else:
            return None
    except:
        return None

def get_canonical_zip_google(addr):
    if(pandas.isnull(addr) or re_noalnum.match(addr)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(addr)
        return None

    try:
        result=geocode_address_google(addr)
        if(result['status']=='OK' and len(result['results'])== 1):
            addr_comps = result['results'][0]['address_components']
            for i in range(0, len(addr_comps)):
                if('postal_code' in addr_comps[i]['types']):
                    return addr_comps[i]['short_name']
        
        return None
    except:
        return None
    
def get_canonical_coords_google(addr):
    if(pandas.isnull(addr) or re_noalnum.match(addr)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(addr)
        return None

    try:
        result=geocode_address_google(addr)
        if(result['status']=='OK' and len(result['results'])== 1):
            latlon = result['results'][0]['geometry']['location']
            if(latlon):
                return(Point(latlon['lng'], latlon['lat']))
        return None
    except:
        return None


In [2]:
def canonicalize_string(text):
    # Remove any leading or trailing whitespace, replace any internal whitespace with single spaces, upcase it all
    ret = re.sub("^\s+|\s+$", "", text)
    ret = re.sub("\s+", " ", ret)
    return ret.upper()

In [3]:
canonicalize_string(" a\t b   ")

'A B'

In [4]:
canonicalize_string("  \t\t\t ")

''

In [3]:
conn = sqlite3.connect('geocoding_cache')
cur = conn.cursor()
cur.execute('SELECT key, value FROM kvs')
google_rows = cur.fetchall()
len(google_rows)

614521

In [4]:
# Key is canonical address, value is a map: {'canonical_in_census_db': True/False,'canonical_in_google_db': True/False, 'other_addr_strings':Set, 'census_value':str, 'google_value':str
canonical_addr_map={}

In [22]:
# Process google DB
count = 0

for (addr,gval) in google_rows:
    caddr = canonicalize_string(addr)
    
    # If there's nothing there, skip it
    if(caddr==""):
        continue
    if(caddr not in canonical_addr_map):
        canonical_addr_map[caddr]={'canonical_in_census_db':False,'canonical_in_google_db': False, 
                                   'other_addr_strings':set(), 
                                   'census_value':None, 'google_value':None}
    # At this point we know there's a (possibly just created) map for caddr in canonical_addr_map.
    # Update it for this entry
    if(caddr in canonical_addr_map):
        if (addr == caddr):
            # This is the canonical entry
            canonical_addr_map[caddr]['canonical_in_google_db']=True
            canonical_addr_map[caddr]['google_value']=gval
        else:
            # This is not the canonical entry
            canonical_addr_map[caddr]['other_addr_strings'].add(addr)
            if(not canonical_addr_map[caddr]['google_value']):
                canonical_addr_map[caddr]['google_value']=gval
            else:
                # Already have a value.  Do we have a canonical one yet?  If not, is this new one the same as what we have?
                if(not canonical_addr_map[caddr]['canonical_in_google_db'] and canonical_addr_map[caddr]['google_value'] != gval):
                    # Compare status and the length of the results, prefer status 'OK' and results length == 1
                    try:
                        stored_res = json.loads(canonical_addr_map[caddr]['google_value'])
                        new_res = json.loads(gval)
                        if(stored_res['status']!='OK' or len(stored_res['results'])!=1):
                            # The old one isn't good.  Try the new one in case it's better.
                            canonical_addr_map[caddr]['google_value']=gval
                            print "Replacing '%s'->'%s' google values do not match" % (addr,caddr)
                    except Exception as e:
                        print "ERROR processing '%s'->'%s' google values do not match\n%s" % (addr,caddr,e)
                           
        
        

Replacing '  ALLEGHENY AVE  '->'ALLEGHENY AVE' google values do not match
Replacing ' ALLEGHENY AVE  '->'ALLEGHENY AVE' google values do not match


In [25]:
num_added = 0
num_already_ok=0

# Find out if any of the entries in canonical_addr_map aren't in google db
for caddr in canonical_addr_map:
    if(not canonical_addr_map[caddr]['canonical_in_google_db']):
        #print "Adding %s (%d others)" % (caddr, len(canonical_addr_map[caddr]['other_addr_strings']))
        cur.execute(('INSERT OR REPLACE INTO kvs (key, value) VALUES (?, ?);'),
                    (caddr, canonical_addr_map[caddr]['google_value']))
        canonical_addr_map[caddr]['canonical_in_google_db']=True
        num_added=num_added+1
    else:
        num_already_ok=num_already_ok+1
print "Added %d, %d already ok" % (num_added, num_already_ok)

Added 294594, 317702 already ok


In [26]:
# Do same processing for census
conn_c = sqlite3.connect('geocoding_cache_census')
cur_c = conn_c.cursor()
cur_c.execute('SELECT key, value FROM kvs')
census_rows = cur_c.fetchall()
len(census_rows)

3379234

In [28]:
# Process census DB
count = 0

for (addr,cval) in census_rows:
    caddr = canonicalize_string(addr)
    
    # If there's nothing there, skip it
    if(caddr==""):
        continue
    if(caddr not in canonical_addr_map):
        canonical_addr_map[caddr]={'canonical_in_census_db':False,'canonical_in_google_db': False, 
                                   'other_addr_strings':set(), 
                                   'census_value':None, 'google_value':None}
    # At this point we know there's a (possibly just created) map for caddr in canonical_addr_map.
    # Update it for this entry
    if(caddr in canonical_addr_map):
        if (addr == caddr):
            # This is the canonical entry
            canonical_addr_map[caddr]['canonical_in_census_db']=True
            canonical_addr_map[caddr]['census_value']=cval
        else:
            # This is not the canonical entry
            canonical_addr_map[caddr]['other_addr_strings'].add(addr)
            if(not canonical_addr_map[caddr]['census_value']):
                canonical_addr_map[caddr]['census_value']=cval
            else:
                # Already have a value.  Do we have a canonical one yet?  If not, is this new one the same as what we have?
                if(not canonical_addr_map[caddr]['canonical_in_census_db'] and canonical_addr_map[caddr]['census_value'] != cval):
                    # Compare status and the length of the results, prefer status 'OK' and results length == 1
                    try:
                        stored_res = json.loads(canonical_addr_map[caddr]['census_value'])
                        new_res = json.loads(cval)
                        if(len(stored_res['result']['addressMatches'])!=1):
                            # The old one isn't good.  Try the new one in case it's better.
                            canonical_addr_map[caddr]['census_value']=cval
                            print "Replacing '%s'->'%s' census values do not match" % (addr,caddr)
                    except Exception as e:
                        print "ERROR processing '%s'->'%s' census values do not match\n%s" % (addr,caddr,e)
                           
        
        

Replacing '    CLIMAX ST  '->'CLIMAX ST' census values do not match
Replacing '   CLIMAX ST  '->'CLIMAX ST' census values do not match
Replacing '   FREELAND ST  '->'FREELAND ST' census values do not match
Replacing '   INDUSTRY ST  '->'INDUSTRY ST' census values do not match
Replacing '   IVY RD  '->'IVY RD' census values do not match
Replacing '   LOYAL WAY  '->'LOYAL WAY' census values do not match
Replacing '   NORWICH  '->'NORWICH' census values do not match
Replacing '   PO BOX 11067        PITTSBURGH PA   15237'->'PO BOX 11067 PITTSBURGH PA 15237' census values do not match
Replacing '   PO BOX 13444        PITTSBURGH PA   15243'->'PO BOX 13444 PITTSBURGH PA 15243' census values do not match
Replacing '   PO BOX 342        VERONA PA   15147'->'PO BOX 342 VERONA PA 15147' census values do not match
Replacing '   PO BOX 42265        PITTSBURGH PA   15203'->'PO BOX 42265 PITTSBURGH PA 15203' census values do not match
Replacing '   PO BOX 801        MONROEVILLE PA   15146'->'PO BOX

Replacing ' PO BOX 62217 PITTSBURGH PA 15241'->'PO BOX 62217 PITTSBURGH PA 15241' census values do not match
Replacing ' PO BOX 66 FOGELSVILLE PA 18051'->'PO BOX 66 FOGELSVILLE PA 18051' census values do not match
Replacing ' PO BOX 66 YOUNGSTOWN PA 15696'->'PO BOX 66 YOUNGSTOWN PA 15696' census values do not match
Replacing ' PO BOX 71254 PITTSBURGH PA 15213'->'PO BOX 71254 PITTSBURGH PA 15213' census values do not match
Replacing ' PO BOX 71283 PITTSBURGH PA 15213'->'PO BOX 71283 PITTSBURGH PA 15213' census values do not match
Replacing ' PO BOX 7137 PITTSBURGH PA 15213'->'PO BOX 7137 PITTSBURGH PA 15213' census values do not match
Replacing ' PO BOX 7139 PITTSBURGH PA 15213'->'PO BOX 7139 PITTSBURGH PA 15213' census values do not match
Replacing ' PO BOX 7142 PITTSBURGH PA 15213'->'PO BOX 7142 PITTSBURGH PA 15213' census values do not match
Replacing ' PO BOX 7202 PITTSBURGH PA 15213'->'PO BOX 7202 PITTSBURGH PA 15213' census values do not match
Replacing ' PO BOX 7254 PITTSBURGH PA

Replacing '1013 ASSOCIATES 1739 E CARSON ST 356 Pittsburgh, PA 15203-1700 '->'1013 ASSOCIATES 1739 E CARSON ST 356 PITTSBURGH, PA 15203-1700' census values do not match
Replacing '1013 ASSOCIATES 1739 E CARSON ST STE 356 Pittsburgh, PA 15203-1700 '->'1013 ASSOCIATES 1739 E CARSON ST STE 356 PITTSBURGH, PA 15203-1700' census values do not match
Replacing '105 NANTUCKET DR PITTSBURGH     PA 15238'->'105 NANTUCKET DR PITTSBURGH PA 15238' census values do not match
Replacing '117 BAKER DR CLAIRTON       PA 15025'->'117 BAKER DR CLAIRTON PA 15025' census values do not match
Replacing '121 EDGEWOOD AVE PITTSBURGH     PA 15218'->'121 EDGEWOOD AVE PITTSBURGH PA 15218' census values do not match
Replacing '124 ROAD D APT B TAMUNING GU 96913-3744 '->'124 ROAD D APT B TAMUNING GU 96913-3744' census values do not match
Replacing '13 ALLEE HENRI MATISSE 77420 CHAMP-SUR-MARNE FRANCE '->'13 ALLEE HENRI MATISSE 77420 CHAMP-SUR-MARNE FRANCE' census values do not match
Replacing '13 ALLEE HENRI MATISSE 

Replacing '4307   BIGELOW BLVD   PITTSBURGH PA 15213'->'4307 BIGELOW BLVD PITTSBURGH PA 15213' census values do not match
Replacing '4307 BIGELOW BLVD PITTSBURGH     PA 15213'->'4307 BIGELOW BLVD PITTSBURGH PA 15213' census values do not match
Replacing '47 ASPERIA STREET REEDY CREEK QLD 4227 AUSTRALIA '->'47 ASPERIA STREET REEDY CREEK QLD 4227 AUSTRALIA' census values do not match
Replacing '499 BLESSING ST PITTSBURGH     PA 15213'->'499 BLESSING ST PITTSBURGH PA 15213' census values do not match
Replacing '50 FT ST  '->'50 FT ST' census values do not match
Replacing '513   CAMBRIDGE ST   PITTSBURGH PA 15213'->'513 CAMBRIDGE ST PITTSBURGH PA 15213' census values do not match
Replacing '6 INKERMAN ST ST KILDA VIC AUSTRALIA 3182 '->'6 INKERMAN ST ST KILDA VIC AUSTRALIA 3182' census values do not match
Replacing '632   LISBON ST   PITTSBURGH PA 15213'->'632 LISBON ST PITTSBURGH PA 15213' census values do not match
Replacing '654 3RD ST  '->'654 3RD ST' census values do not match
Replacin

In [30]:
num_added = 0
num_already_ok=0

# Find out if any of the entries in canonical_addr_map aren't in census db
for caddr in canonical_addr_map:
    if(canonical_addr_map[caddr]['census_value']):
        if(not canonical_addr_map[caddr]['canonical_in_census_db']):
            #print "Adding %s (%d others)" % (caddr, len(canonical_addr_map[caddr]['other_addr_strings']))
            cur_c.execute(('INSERT OR REPLACE INTO kvs (key, value) VALUES (?, ?);'),
                          (caddr, canonical_addr_map[caddr]['census_value']))
            canonical_addr_map[caddr]['canonical_in_census_db']=True
            num_added=num_added+1
        else:
            num_already_ok=num_already_ok+1
print "Added %d, %d already ok" % (num_added, num_already_ok)

Added 676576, 2030685 already ok


In [ ]:
# Try to save out canonical_addr_map
import pickle
pickle_file_path = 'assessments/canonical_addr_map.pickle'

In [ ]:
# Load in canonical_addr_map
with open(pickle_file_path, 'rb') as handle:
    canonical_addr_map = pickle.load(handle)


In [31]:
# Save out canonical_addr_map
with open(pickle_file_path, 'wb') as handle:
    pickle.dump(canonical_addr_map, handle, protocol=pickle.HIGHEST_PROTOCOL)